In [5]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import List, Dict, Any
from datetime import datetime, timedelta

# .env 파일 로드
load_dotenv()

# Gemini API 설정
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY_JY")
if not GEMINI_API_KEY:
    raise ValueError("GEMINI_API_KEY_JY가 .env 파일에 설정되지 않았습니다.")
else:
    print("GEMINI_API_KEY_JY가 설정되었습니다.")

GEMINI_API_KEY_JY가 설정되었습니다.


In [2]:
# GEMINI 활성화

keyword_gemini = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash-preview-09-2025",
    google_api_key = GEMINI_API_KEY,
    temperature = 0.4
)

news_gemini = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash-preview-09-2025",
    google_api_key = GEMINI_API_KEY,
    temperature = 0.1
)

E0000 00:00:1761898545.856536 3122233 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1761898545.861991 3122233 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [ ]:
# GEMINI 보고 뉴스 키워드 만들어달라고 하기 

class SearchKeywords(BaseModel):
    reasons: List[str] = Field(description="각 검색어 선택 이유들")
    keywords: List[str] = Field(description="사용자가 요청한 기업 주가 분석 예측에 유용한 검색어 리스트들")
    
parser = JsonOutputParser(pydantic_object=SearchKeywords)

prompt = PromptTemplate.from_template(
    """당신은 금융 분석 전문가입니다. 
사용자가 원하는 기업의 주가를 예측하기 위해 뉴스 검색에 사용할 최적의 검색어를 생성해주세요.
답변은 반드시 Json 형식으로 답변해주세요.

## 답변시 고려해야 할 사항
    1. 현재 날짜: {current_date}
    2. 3~5개 정도의 키워드 생성

{format_instructions}

사용자가 요청한 기업: ```{company_name}```
    """
)

chain = prompt | keyword_gemini | parser

company_name = "NVIDIA"

result = chain.invoke({"format_instructions": parser.get_format_instructions(), "current_date": datetime.now().strftime("%Y-%m-%d"), "company_name": company_name})

print(result)

{'keywords': ['NVIDIA Data Center Revenue Guidance', 'NVIDIA Next Generation GPU Roadmap', 'AI GPU Supply Chain Outlook', 'NVIDIA AMD Intel AI Competition', 'NVIDIA China Export Regulations Impact']}


In [7]:
# Test News가 있다고 가정

news1 = """코스피 4100선 안착…현대차 시총 5위로 복귀
2025년 10월 31일 17:07 GMT+9


NVDA
−2.00%
코스피지수가 4100선에 안착했다. 4000선을 넘긴 지 4거래일 만이다. 젠슨 황 엔비디아 최고경영자(CEO)와 ‘인공지능(AI) 동맹’을 맺은 현대자동차그룹, 삼성전자 주가가 크게 상승하며 지수를 견인했다.

31일 코스피지수는 0.50% 오른 4107.50에 거래를 마쳤다. 이날 증시의 주인공은 자동차주였다. 현대차는 9.43% 오른 29만원에 거래를 마치며 52주 신고가를 경신했다. 두산에너빌리티를 제치고 시가총액 5위 자리에 안착했다. 기아도 3.18% 상승했다. 현대위아(4.46%), 현대모비스(2.93%) 등도 일제히 올랐다.

한·미 관세협상 타결로 대미 자동차 수출 관세가 기존 25%에서 15%로 크게 낮아진 데다 엔비디아와의 ‘AI 동맹’이 가시화하면서 관심을 끌었다. 현대차그룹은 엔비디아의 최신 그래픽처리장치(GPU) 5만 개를 공급받고 이를 활용해 통합 AI 모델 개발을 추진한다는 계획이다. 또한 국내 피지컬 AI 발전을 위해 엔비디아와 약 30억달러를 투자하기로 했다. 미국 입항 수수료 부과가 유예되면서 현대글로비스도 10.41% 급등하는 등 현대차그룹 계열사 주가가 일제히 강세를 보였다.

엔비디아와 손잡고 반도체 개발·생산 경쟁력을 끌어올리기 위한 반도체 AI 팩토리 구축에 나선다고 발표한 삼성전자도 이날 3.27% 오른 10만7500원에 거래를 마감했다.

전날 황 CEO가 “한국에 좋은 소식이 있고 힌트는 AI, 로보틱스와 관련된 것”이라고 발언한 영향으로 로봇 관련주도 대폭 뛰었다. 레인보우로보틱스는 24.01% 급등한 43만6500원에 거래를 마쳤다.

2차전지주는 일제히 약세를 보였다. ‘에너지저장장치(ESS) 시장이 크게 성장하고 있지만 북미 전기차 수요 둔화의 충격을 상쇄하긴 어렵다’는 비관론이 커진 결과다.

심성미 기자"""


news2 = """삼성전자, 엔비디아와 함께 ‘업계 최고 수준 반도체 AI 팩토리’ 구축
2025/10/31

AUDIO Play
Download
공유 레이어 열기
인쇄하기
업계 최고 수준 반도체 AI 팩토리 구축… 글로벌 제조 패러다임 전환
삼성전자는 31일 엔비디아와의 전략적 협력을 통해 반도체 제조의 새로운 패러다임을 제시할 ‘반도체 AI 팩토리’를 구축한다고 밝혔다.

삼성전자는 메모리∙시스템반도체∙파운드리를 아우르는 업계 최대 수준의 반도체 제조 인프라를 갖춘 종합 반도체 기업이다.

삼성전자는 종합반도체 기업으로서의 역량과 엔비디아의 GPU 기반 AI 기술의 시너지를 통해 업계 최고 수준의 ‘반도체 AI 팩토리’를 구축, 반도체를 비롯한 글로벌 제조 산업의 패러다임 전환을 주도해 나갈 계획이다.

이를 위해 삼성전자는 향후 수년간 5만개 이상의 엔비디아 GPU를 도입해 AI 팩토리 인프라를 확충하고, 엔비디아의 시뮬레이션 라이브러리 옴니버스(Omniverse) 기반 디지털 트윈 제조 환경 구현을 가속화할 예정이다.

삼성전자가 추진하는 AI 팩토리는 반도체 제조 과정에서 생성되는 모든 데이터를 실시간으로 수집해 스스로 학습하고 판단하는 지능형 제조 혁신 플랫폼이다.

AI 팩토리는 ▲설계 ▲공정 ▲운영 ▲장비 ▲품질관리 등 반도체 설계와 생산을 아우르는 모든 과정에 AI를 적용해 스스로 분석∙예측∙제어하는 ‘생각하는’ 제조 시스템이 구현된 스마트 공장이다.

삼성전자는 AI 팩토리 구축을 통해 차세대 반도체 개발∙양산 주기를 단축하고, 제조 효율성과 품질 경쟁력을 혁신적으로 강화할 계획이다.

HBM4 등 차세대 메모리 기술력으로 AI 생태계 혁신에 기여할 계획
삼성전자는 AI 팩토리 구축과 함께 엔비디아에 ▲HBM3E ▲HBM4 ▲GDDR7 ▲SOCAMM2 등 차세대 메모리와 파운드리 서비스도 공급해 글로벌 AI생태계에서 삼성전자와 엔비디아의 위상을 더욱 공고히 할 계획이다.

삼성전자는 이미 공급 중인 메모리 제품뿐만 아니라 성능과 에너지 효율을 대폭 향상시킨 HBM4 공급을 엔비디아와 긴밀하게 협의 중이다.

삼성전자 HBM4의 경우, 1c(10나노급 6세대) D램 기반에 4나노 로직 공정을 적용하고 설계를 최적화해 JEDEC 표준(8Gbps) 및 고객 요구를 상회하는 11Gbps 이상의 성능을 구현했다.

삼성전자 HBM4는 초고대역폭과 저전력 특성을 바탕으로 AI 모델 학습과 추론 속도를 높여 엔비디아의 AI 플랫폼 성능 향상에 핵심적으로 기여할 것으로 전망된다.

현재 삼성전자는 글로벌 전 고객사에게 HBM3E를 공급하고 있으며, HBM4도 샘플을 요청한 모든 고객사에 샘플 출하를 완료한 뒤 고객사 일정에 맞춰 양산 출하를 준비하고 있다.

삼성전자는 급격하게 증가하고 있는 고객사 HBM4 수요에 차질 없이 대응하기 위해 선제적으로 설비 투자를 진행할 예정이다.

삼성전자는 HBM 외에도 업계 최초로 개발한 고성능 그래픽 D램(GDDR7)과 차세대 저전력 메모리 모듈 SOCAMM2 공급도 협의 중이며, 파운드리 분야에서도 협력을 강화할 계획이다.

삼성전자, 엔비디아 플랫폼 활용한 ‘반도체 AI 팩토리’ 구축 노하우 축적
삼성전자는 이미 일부 공정에서 엔비디아의 플랫폼을 활용해 반도체 AI 팩토리의 기반을 다져왔다. 이번 전략적 협력을 통해 양사는 축적된 협업의 노하우를 활용, 혁신을 가속화한다는 계획이다.

삼성전자는 엔비디아의 AI 컴퓨팅 기술인 ▲쿠리소(cuLitho) ▲쿠다-X(CUDA-X)를 도입해, 미세 공정에서 발생할 수 있는 회로 왜곡을 AI가 실시간으로 예측∙보정함으로써 공정 시뮬레이션 속도를 기존보다 20배 향상시키고 설계 정확도와 개발 속도를 동시에 향상시켰다.

또한, 생산 설비의 실시간 분석∙이상 감지∙자동 보정이 가능한 통합 제어 체계를 구축했으며, 옴니버스 기반의 ‘Digital Twin(디지털 트윈)’을 통해 가상 공간에서 ▲설비 이상 감지 ▲고장 예측 ▲생산 일정 최적화 등도 구현 중이다.

향후 삼성전자는 AI 팩토리 인프라 구축과 관련 노하우를 한국 뿐 아니라 미국 테일러 등 해외 주요 생산 거점에까지 확장해, 글로벌 반도체 공급망 전체의 지능화와 효율화를 완성할 계획이다.

삼성전자 AI 팩토리, AI 중심 국가 제조 생태계 질적 성장 견인
삼성전자의 AI 팩토리 구축은 단순한 제조 혁신을 넘어, 국가 반도체 생태계의 질적 성장을 견인하고 국가 제조 산업이 AI 중심으로 전환되는데 촉매 역할을 할 것으로 기대된다.

삼성전자는 AI 팩토리 구축을 위해 국내 팹리스, 장비, 소재 기업들과 전방위적으로 협력을 확대해 나갈 계획이다.

또한 향후 AI 팩토리가 협력 중소 기업들의 AI 역량 강화를 견인할 수 있는 플랫폼이 될 수 있도록 발전시켜 나갈 것이다.

삼성전자는 AI 팩토리를 중심으로 엔비디아와 함께 국내외 파트너사 및 EDA 기업들과 차세대 반도체 설계 도구를 공동 개발하고 AI 기반 반도체 제조 표준을 선도해 AI 생태계 발전에 이바지할 계획이다.

삼성전자는 중소기업의 제조 경쟁력 향상을 지원하기 위해 AIㆍ데이터 기술을 활용해 기존 공장을 지능형 스마트 공장으로 고도화하는 ‘스마트공장3.0’ 사업도 이미 전개하고 있다.

삼성전자의 이러한 노력들은 대한민국이 글로벌 AI 패러다임을 선도하고 글로벌 3대 AI 강국으로 도약하는데 핵심적인 역할을 할 것으로 기대된다.

AI 모델∙휴머노이드 로봇∙AI-RAN 기술에서도 삼성-엔비디아 협업 강화
삼성전자는 AI 모델과 휴머노이드 로봇 기술을 고도화하고 관련 기술을 AI 팩토리를 포함한 다양한 분야로 확대해, 생성형 AI·로보틱스·디지털 트윈 등을 아우르는 차세대 AI 생태계를 구축해 나갈 계획이다.

삼성전자의 AI 모델은 엔비디아 GPU상에서 메가트론(Megatron) 프레임워크를 사용하여 구축됐다. 고도화된 추론 능력을 기반으로 실시간 번역, 다국어 대화, 지능형 요약 등에서 탁월한 성능을 발휘한다.

또한 삼성전자는 다양한 제품의 제조 자동화 및 휴머노이드 로봇 분야 전반에서 NVIDIA RTX PRO 6000 블랙웰 서버 에디션 플랫폼을 활용해 지능형 로봇의 상용화와 자율화 기술 고도화를 추진 중이다.

삼성전자는 엔비디아의 다양한 AI 플랫폼을 기반으로 가상 시뮬레이션 데이터와 실제 로봇 데이터를 연결해, 현실 세계를 인식하고 스스로 판단·작동할 수 있는 로봇 플랫폼도 구현하고 있다.

아울러 엔비디아의 젯슨 토르(Jetson Thor) 로보틱스 플랫폼을 활용하여 지능형 로봇의 AI 추론, 작업 수행, 안전 제어 기술을 강화하고 있다.

또한, 삼성전자는 엔비디아 및 국내 산·학·연과 차세대 지능형 기지국(AI-RAN) 기술 연구 및 실증을 위한 MoU를 체결했다.

지능형 기지국(AI-RAN)은 네트워크 및 AI 기술을 융합해 차세대 AI 로봇 등 피지컬 AI 및 새로운 서비스의 구현을 지원하는 차세대 통신 기술이다. 이는 로봇, 드론, 산업현장의 자동화 장비 등 피지컬 AI가 통신망에서 실시간으로 동작, 센싱, 데이터 연산 및 추론을 가능하게 해 피지컬 AI 도입과 확산에 필수적인 신경망 역할을 한다.

삼성전자는 지난해 엔비디아와 협력해 지능형 기지국(AI-RAN) 기술 검증에 성공한 바 있으며, 이번 MoU 체결을 통해 AI 및 소프트웨어 기반 협력을 지속해 나갈 예정이다.

25년 협력의 결실, AI 반도체 동맹으로 진화
삼성전자는 엔비디아 그래픽카드에 D램을 공급한 것을 시작으로 파운드리 분야까지 파트너십을 지속적으로 강화해 왔다.

이번 프로젝트는 25년 이상 이어온 양사의 기술 협력이 맺은 결실로, 업계 최고 수준의 반도체 AI 팩토리 구현이라는 상징적 의미를 담고 있다."""

news3 = """엔비디아, 세계 최초로 시총 5조달러 돌파
젠슨 황 CEO사진 출처,Reuters
사진 설명,2024년 대만에서 열린 행사에 참석한 젠슨 황 엔비디아 CEO
기사 관련 정보
기자,다니엘 카예
기자,비즈니스 전문기자
2025년 10월 30일
미국 반도체 대기업 '엔비디아'가 지난 29일(현지시간) 세계 최초로 시가총액 5조달러(약 7100조원)를 돌파했다.

틈새시장을 공략하던 그래픽 칩 제조사였던 엔비디아는 인공지능(AI) 분야의 성장 기대감 속 AI 반도체의 거인으로 발돋움했고, 주가도 폭발적으로 상승하며 기록을 세웠다.

2023년 6월 처음으로 시가총액 1조달러를 달성했으며, 불과 3개월 전 4조달러를 넘어섰다.

29일 오전, 엔비디아의 주가는 지정학적 긴장 지점인 중국 내 매출에 대한 시장의 낙관론에 힘입어 5.6%까지 상승하며 212달러를 넘어섰다.

세계 1위의 시가총액을 자랑하는 기업이자, AI 투자 열풍의 최대 수혜자로 손꼽히는 엔비디아는 기술 분야에서 다른 경쟁사들을 압도적으로 앞서고 있다.

엔비디아의 반도체칩이 AI 열풍의 핵심 동력인 가운데 엔비디아는 오픈AI, 오라클 등 주요 AI 기업들과 계약을 체결한 상태다.

세계은행(WB)에 따르면 엔비디아의 현재 기업 가치는 미국과 중국을 제외한 전 세계 국가의 국내총생산(GDP)을 뛰어넘었으며, S&P500 지수 내 산업 전체보다도 크다.

마이크로소프트와 애플 역시 최근 시총 4조달러 고지를 돌파하며 월가의 AI 투자 낙관론에서 촉발된 기술주 상승세는 더욱 공고해지는 모습이다.

올해 미국 증시의 놀라운 상승분 중 80%는 AI 관련 기업들이 차지했다.

그러나 기술 기업들의 주가가 연일 최고치를 경신하면서 AI 버블 및 관련 기업들의 과대평가 여부에 대한 우려도 계속 커지고 있다.

영란은행, 국제통화기금(IMF), 제이미 다이먼 JP모건체이스 CEO도 이 같은 경고를 내놓았다. 특히 다이먼 CEO는 최근 BBC와의 인터뷰에서 "사람들이 불확실성을 더 크게 인식해야 한다"고 말했다.

'AJ벨'의 대니 휴슨 재무 분석 책임자는 엔비디아의 시총 5조달러 돌파에 대해 "사람의 머리로는 제대로 감을 잡기도 어려울 정도로 거대한 금액"이라고 표현했다.

이어 "물론 그렇다고 AI 버블에 대한 우려가 사라지는 것은 아니지만, 시장은 (우려 목소리에도 불구하고) 앞으로 나아가려는 듯한 모양새"라고 덧붙였다.

일부 회의론자들은 AI 기술 기업들의 폭발적인 가치 상승이 적어도 부분적으로는 자신들이 '금융 공학'이라고 부르는 것의 결과일 수 있다며 공개적으로 의문을 제기하기도 한다.

주요 AI 기업들은 서로에게 투자하며 복잡하게 얽힌 거래망을 형성해왔는데. 이러한 구조에 대해 규제 당국과 시장은 더욱 자세히 지켜보고 있다.

일례로 지난 2022년 '챗GPT'로 AI 대중화에 앞장선 오픈AI사는 지난달 엔비디아로부터 1000억달러의 투자를 확보했다."""

news4 = """(LEAD) [APEC 2025] Nvidia to deploy 260,000 GPUs to build AI factories with Samsung, Hyundai in S. Korea
Chang Dong-woo, 장동우
Chang Dong-woo
All News 15:27 October 31, 2025
SHARE LIKE SAVE PRINT FONT SIZE
(ATTN: UPDATES from para 10; ADDS photo)
By Chang Dong-woo

GYEONGJU, South Korea, Oct. 31 (Yonhap) -- Nvidia Corp. plans to deploy up to 260,000 graphics processing units (GPUs) in South Korea in partnership with the government and major companies here to build large-scale artificial intelligence (AI) factories in the country, the U.S. chip giant said Friday.

The initiative will involve the South Korean government and major industry leaders, including Samsung Electronics. Co., SK Group, Hyundai Motor Group and Naver Cloud Corp., that will each establish their own AI computing centers powered by Nvidia's latest Blackwell GPUs.

Under the plan, which Nvidia shared in a preview media briefing on Thursday, the Seoul government will allocate up to 50,000 Nvidia GPUs to support the creation of a national "sovereign AI" platform designed to train large language models (LLMs) optimized for Korean language and industrial applications.

A GPU is a high-performance semiconductor chip designed to handle many calculations simultaneously and is widely used to train and run AI models.

Nvidia Corp. CEO Jensen Huang appears on stage at a gaming event hosted by the U.S. chip giant at COEX in Seoul on Oct. 30, 2025. (Pool photo) (Yonhap)
Nvidia Corp. CEO Jensen Huang appears on stage at a gaming event hosted by the U.S. chip giant at COEX in Seoul on Oct. 30, 2025. (Pool photo) (Yonhap)

Samsung, SK and Hyundai Motor will each deploy up to 50,000 GPUs, while Naver Cloud will install around 60,000 units, bringing the total to 260,000.

Nvidia said the Blackwell-based infrastructure will boost South Korea's total installed AI GPU capacity from about 65,000 to more than 300,000 units, positioning the country as one of the world's largest hubs for AI computing outside the United States.

Jensen Huang, Nvidia's chief executive officer (CEO), plans to share further details of the initiative on the sidelines of the Asia-Pacific Economic Cooperation (APEC) CEO Summit under way in Gyeongju, the host city of this year's multilateral gathering of regional leaders, later in the day.

Huang arrived in South Korea on Thursday and met with Hyundai Motor Group Executive Chair Euisun Chung and Samsung Electronics Chairman Lee Jae-yong in Seoul.

The Nvidia chief said his visit came at the invitation of President Lee Jae Myung, and said there will be an announcement on details of his company's partnership with South Korean companies.

GalleryPresident Lee Jae Myung (R) shakes hands with Nvidia Corp. CEO Jensen Huang during their meeting on the sidelines of the Asia-Pacific Economic Cooperation summit in the southeastern city of Gyeongju, 275 kilometers southeast of Seoul, on Oct. 31, 2025. (Yonhap)
President Lee Jae Myung (R) shakes hands with Nvidia Corp. CEO Jensen Huang during their meeting on the sidelines of the Asia-Pacific Economic Cooperation summit in the southeastern city of Gyeongju, 275 kilometers southeast of Seoul, on Oct. 31, 2025. (Yonhap)

Financial terms of the arrangements were not disclosed.

Samsung will use the new AI factory to accelerate semiconductor manufacturing through digital twin simulations and smart robotics, employing Nvidia's NeMo, CUDA-X and Omniverse platforms, according to the company.

SK Group said it will design a similar facility to support semiconductor research and cloud-based AI applications, while SK Telecom Co. will provide sovereign industrial cloud services using Nvidia's RTX Pro 6000 Blackwell Server Edition GPUs.

Hyundai Motor will use the 50,000 Blackwell GPUs to train large AI models for autonomous driving, robotics and smart factory operations, with plans to co-invest around US$3 billion with the government to build a "physical AI" ecosystem.

With the Blackwell GPUs, Naver Cloud plans to provide the computing backbone for domestic AI foundation models.

LG Group will also partner with Nvidia in the fields of robotics and medical technology.

In particular, LG and Nvidia plan to advance robotics capabilities and support an AI-driven cancer diagnosis research ecosystem for startups and academia using LG AI Research's large-scale AI model EXAONE.

In addition, Nvidia plans to work with Samsung, SK Telecom, KT Corp., LG Uplus Corp. and Yonsei University to develop AI-native 6G radio access networks (AI RAN), a next-generation wireless system.

odissy@yna.co.kr
(END)"""

news5 = """Nvidia, Samsung, Hyundai chiefs’ gathering in Seoul gains attention
2025.10.31 11:07:56
printfacebooktwitterlinkedin
(Lee Seung-hwan)사진 확대
(Lee Seung-hwan)
The chiefs of Nvidia Corp., Samsung Electronics Co., and Hyundai Motor Group gathered in Seoul on Thursday, at a fried chicken restaurant where they discussed ties over beer.

The gathering – attended by Nvidia Chief Executive Officer Jensen Huang, Samsung Electronics Executive Chairman Jay Y. Lee, and Hyundai Motor Group Executive Chair Euisun Chung – took place at Kkanbu Chicken in southern Seoul.

Huang appeared relaxed in a leather jacket and T-shirt, as did the two Korean executives. The restaurant was reportedly chosen by Nvidia to reflect the companies’ growing partnership.

Kkanbu is a Korean term meaning a close friend or companion.

Samsung supplies high-bandwidth memory (HBM) for Nvidia’s artificial intelligence (AI) chips, while Hyundai Motor uses Nvidia’s autonomous driving platform to develop software-defined vehicles (SDVs).

Huang arrived in Seoul earlier in the day for his first official visit in 15 years to attend the Asia-Pacific Economic Cooperation (APEC) CEO Summit in Gyeongju, North Gyeongsang Province.

Seated by the window, the three executives shared chicken and beer, shook hands, and raised their glasses together.

The table featured dishes like crispy boneless chicken and garlic soy chicken, each priced around 20,000 won ($13.98).

Huang pulled out bottles of premium Japanese whisky, Hakushu 25 Years, signed them, and gave one each to Lee and Chung.

He also presented them with Nvidia’s newly unveiled miniature AI supercomputer, the DGX Spark. The unit gifted to Lee carried a message: “Jay. To our Partnership and Future of the World.” Samsung’s memory products are used inside the DGX Spark.

While dining, Huang invited nearby diners to join a toast, greeted fans gathered outside, and even handed out chicken. Asked by reporters why he chose a fried chicken joint for the dinner, he said he loved fried chicken and beer, especially when shared with friends.

When asked about the meeting, Lee said sharing good food with friends was happiness, noting that the tariff issue had been resolved.

Chung also thanked government officials and said businesses should now build on that progress.

After dining, Lee and Chung joined Huang on stage at Nvidia’s GeForce Gamer Festival, held to mark the 25th anniversary of the GeForce graphics card.

Huang recalled that his first trip to Korea was in 1996, prompted by a letter from the late Lee Kun-hee, the former Samsung chairman and father of Jay Y. Lee.

Lee noted that Nvidia had used Samsung’s DDR DRAM in the original GeForce 256, launched 25 years ago. He said he attended not only because Nvidia is an important client and strategic partner, but also because “Jensen is one of the greatest innovators of our time and a business leader I deeply respect.”

Chung said Hyundai plans to deepen its collaboration with Nvidia in robotics and automotive technology.

Huang, Lee, and Chung are expected to meet again on Friday at the APEC CEO Summit in Gyeongju, where Huang will deliver a keynote address.

Industry sources say they will likely be joined by SK Group Chairman Chey Tae-won and Naver Corp. founder and board chair Lee Hae-jin to discuss major partnerships, including large-scale purchases of Nvidia GPUs.

By Lee Jin-han, Park Min-ki, Lee Deok-joo and Chang Iou-chung
[ⓒ Pulse by Maeil Business News Korea & mk.co.kr, All rights reserved]"""

In [ ]:
# 뉴스들 내용을 분석
news_list = [news1, news2, news3, news4, news5]

class NewsAnalysis(BaseModel):
    positive_points: List[str] = Field(description="긍정적인 내용들")
    negative_points: List[str] = Field(description="부정적인 내용들")
    reference_points: List[str] = Field(description="참고할만한 내용들")
    
parser = JsonOutputParser(pydantic_object=NewsAnalysis)

prompt = PromptTemplate.from_template(
    """너는 뉴스들을 보고 정리해주는 전문가야.
아래에 주어진 뉴스들을 보고 Json 형식으로 분류해줘
    
    
    """
)
